### Notebook for parsing porn stars names and their photos

#Get porn stars data from *pornhub.com*

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
from tqdm import tqdm

In [ ]:
first_page = "https://rt.pornhub.com/pornstars?page=1"
max_page = 1754

In [ ]:
def get_pstars(page) -> list:
    page_stars = []
    
    req = requests.get(f"https://rt.pornhub.com/pornstars?page={page}")
    soup = BeautifulSoup(req.text, 'html.parser')
    
    for star in soup("li", "pornstarLi"):
        one_star = {}
        # get name
        one_star['name'] = star(class_="js-mxp")[0]["data-mxptext"]
        # get number of videos and views
        vp = star(class_="videosNumber")[0].text
        one_star['videos'] = vp.split("\t\t\t\t")[0].strip(" ")
        one_star['views'] = vp.split("\t\t\t\t")[1].strip(" ")
        page_stars.append(one_star)
        
    return page_stars

In [ ]:
all_pages = []

In [ ]:
for page in tqdm(range(1, max_page+1), position=0, leave=True):
    all_pages += get_pstars(page)
    time.sleep(2)

100%|██████████| 1754/1754 [1:25:53<00:00,  2.94s/it]


In [ ]:
ph_stars = pd.DataFrame(all_pages)

In [ ]:
ph_stars

,name,videos,views
0,Lana Rhoades,391 Видео,1.2B показов
1,Abella Danger,996 Видео,1.4B показов
2,Eva Elfie,149 Видео,716M показов
3,Riley Reid,834 Видео,1.5B показов
4,Mia Malkova,533 Видео,706M показов
...,...,...,...
17724,Callie Bond,0 Видео,0 показов
17725,Dawid White,0 Видео,0 показов
17726,Mirage806,0 Видео,0 показов
17727,Jon Davids,1 Видео,23.7K показов


In [ ]:
# get number of videos
ph_stars['videos'] = ph_stars['videos'].apply(lambda x: int(re.match(r"(\d{1,8}).*", x).group(1)))

# clean views
ph_stars['views'] = ph_stars['views'].apply(lambda x: re.match(r"(.*)\s.*", str(x)).group(1))

#convert views to numbers
def get_number(x):
    number = float(re.match(r".*[^a-zA-Z]", x).group(0))
    try:
        r = {"B":1000000000, "M":1000000, "K":1000}[re.match(r".*([B|M|K])", x).group(1)]
    except:
        r = 1
    return number * r

ph_stars['views'] = ph_stars['views'].apply(lambda x: get_number(x))

In [ ]:
ph_stars.to_csv("ph_stars.csv")

In [ ]:
ph_stars[ph_stars['videos']>10]

,name,videos,views
0,Lana Rhoades,391,1.200000e+09
1,Abella Danger,996,1.400000e+09
2,Eva Elfie,149,7.160000e+08
3,Riley Reid,834,1.500000e+09
4,Mia Malkova,533,7.060000e+08
...,...,...,...
16787,Beau Jingles,11,6.600000e+03
16852,Arcanjo Amaro,21,6.950000e+05
16940,Labelly Sandorram,12,4.520000e+05
17090,Sin-A-Men,12,4.120000e+05


In [ ]:
lst_pa = ph_stars[ph_stars['videos']>10]['name']
lst_pa = list(dict.fromkeys(lst_pa))
len(lst_pa)

7898

In [ ]:
ph_stars.drop_duplicates('name', inplace=True)
ph_stars.sort_values(by='views', ascending=False, inplace=True)

In [ ]:
ph_stars

,name,videos,views
122,Tony Profane,543,2.800000e+09
8,Johnny Sins,991,2.300000e+09
11,Alex Adams,319,1.800000e+09
53,James Deen,1075,1.600000e+09
3,Riley Reid,834,1.500000e+09
...,...,...,...
17624,Taylor J. Morgan,0,0.000000e+00
17625,Christy Body,0,0.000000e+00
17626,Janet Saisque,0,0.000000e+00
14219,Skyla Paris,0,0.000000e+00


In [ ]:
ph_stars.to_csv("ph_stars_p.csv")

#800 pornhub best

In [ ]:
req = requests.get("https://rt.pornhub.com/pornstars/top")
soup = BeautifulSoup(req.text, 'html.parser')

b800ph = []
base_url = 'https://rt.pornhub.com'

In [ ]:
for i in soup(class_='index-title')[1:]:
    name = re.match(r'.*">(.*)</a></li>', str(i)).group(1)
    href = base_url + re.match(r'.*href="(.*)".*', str(i)).group(1)
    b800ph.append((name, href))

In [ ]:
b800ph_df = pd.DataFrame(b800ph, columns=['name', 'url'])

In [ ]:
b800ph_df

,name,url
0,Lana Rhoades,https://rt.pornhub.com/pornstar/lana-rhoades
1,Abella Danger,https://rt.pornhub.com/pornstar/abella-danger
2,Eva Elfie,https://rt.pornhub.com/pornstar/eva-elfie
3,Riley Reid,https://rt.pornhub.com/pornstar/riley-reid
4,Mia Malkova,https://rt.pornhub.com/pornstar/mia-malkova
...,...,...
795,Angel Smalls,https://rt.pornhub.com/pornstar/angel-smalls
796,Megan Marx,https://rt.pornhub.com/pornstar/megan-marx
797,Eveline Dellai,https://rt.pornhub.com/pornstar/eveline-dellai
798,Sean Lawless,https://rt.pornhub.com/pornstar/sean-lawless


In [ ]:
b800ph_df.to_csv("/content/drive/MyDrive/python/PronStarSimilarity/b800ph.csv")

get photos 800 pornhub best

In [ ]:
index = 6
while index < len(b800ph_df):
    req = requests.get(b800ph_df['url'][index])
    soup = BeautifulSoup(req.text, 'html.parser')
    try:
        response = requests.get(soup(class_='previewAvatarPicture')[0](class_='jcrop-preview')[0]['src'])
        file = open(f"/content/drive/MyDrive/python/PronStarSimilarity/pornhub_800best/{b800ph_df['name'][index]}_b800.png", "wb")
        file.write(response.content)
        file.close()
    except:
        response = requests.get(soup(class_='thumbImage')[0]('img')[0]['src'])
        file = open(f"/content/drive/MyDrive/python/PronStarSimilarity/pornhub_800best/{b800ph_df['name'][index]}_b800.png", "wb")
        file.write(response.content)
        file.close()        

    time.sleep(2)
    index += 1

#Get photos of actors from bing.com *!not used*

Not used because of loading many non relevant photos

In [ ]:
!pip install bing-image-downloader

In [ ]:
from bing_image_downloader import downloader
from IPython.display import clear_output 
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dir_images = "/content/drive/MyDrive/python/PronStarSimilarity/porn_stars"

Mounted at /content/drive


In [ ]:
# load database to google drive
ph_stars.to_csv("/content/drive/MyDrive/python/PronStarSimilarity/ph_starts_p_all.csv", index=False)

In [ ]:
ph_stars['name'] = ph_stars['name'].apply(lambda x: re.sub(r'\s+', ' ', x).strip(" "))
ph_stars = ph_stars[ph_stars['name'].str.contains(" ")]

In [ ]:
ph_stars_f = ph_stars[ph_stars['videos']>50]
ph_stars_f

,name,videos,views
0,Tony Profane,543,2.800000e+09
1,Johnny Sins,991,2.300000e+09
2,Alex Adams,319,1.800000e+09
3,James Deen,1075,1.600000e+09
4,Riley Reid,834,1.500000e+09
...,...,...,...
12471,Gorgeousleigh Chaotic,58,6.390000e+04
12593,Eve X,56,5.690000e+04
12608,Mistress Irony,52,5.630000e+04
13025,Jackie Brown,162,3.680000e+04


In [ ]:
# load filtered database to google drive
ph_stars_f.to_csv("/content/drive/MyDrive/python/PronStarSimilarity/ph_stars_f.csv", index=False)

In [ ]:
for star in tqdm(ph_stars_f['name'][909:], position=True, leave=True):
    search_request = star + " porn star"
    downloader.download(search_request, limit=5,  output_dir=dir_images, adult_filter_off=False, force_replace=False, timeout=60, verbose=True)
    clear_output()
    time.sleep(2)

  5%|▌         | 109/2155 [11:17<3:31:58,  6.22s/it]


KeyboardInterrupt: ignored

# Get porn star photos in IAFD (internet adult film database)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

In [ ]:
ps_db['iafd_links'] = ph_stars['name'].apply(lambda x: f'https://www.iafd.com/person.rme/perfid={str(x).lower().replace(" ", "")}/gender=f/{str(x).lower().replace(" ", "-")}.htm')
ps_db['load'] = ph_stars['iafd_links']

In [ ]:
# Initialization of a new session at each iteration is needed to bypass the site captcha
index = 0
while index < len(ps_db):
    try:
        chrome_options = Options()
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--ignore-certificate-errors')
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome() #options=chrome_options
        driver.get(ps_db['iafd_links'][index])
        timeout = 10
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="headshot"]/img'))
        WebDriverWait(driver, timeout).until(element_present)
        time.sleep(5)
        response = requests.get(driver.find_elements_by_xpath('//*[@id="headshot"]/img')[0].get_attribute("src"))
        
        file = open(f"./ps_photos/{ps_db['name'][index]}.jpeg", "wb")
        file.write(response.content)
        ps_db.iloc[index,-1] = 'loaded'
        file.close()
        
        driver.close()
        ps_db.to_csv('iafd_links.csv', index=False)
        time.sleep(3)
    except:
        driver.close()
        time.sleep(3)
        pass
    index += 1